In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
%%time
bdinfo_df = pd.read_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_4.csv')
print(bdinfo_df.shape)
bdinfo_df.head()

<string>:2: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.


(935428, 50)
Wall time: 4.39 s


,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자
0,11440-21639,집합,표제부,0,주건축물,177.62,105.300,59.28,393.130,385.210,216.87,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트 평스라브,12.00,4,1,0,0,0,0.0,393.130,0,0.0,0,0.0,0,0.00,4,46.0,20040323.0,NaN,0.0,0,NaN,0,NaN,0,20101230,201412,서울특별시 마포구 서교동 484-16,4,일반,20040323.0,20040323.0,20101230,20170209
1,11590-100182663,집합,표제부,1,주건축물,389.48,232.420,59.67,947.740,659.890,169.43,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,11.26,4,1,0,0,0,0.0,947.740,0,0.0,0,0.0,8,287.85,2,11.5,20080826.0,NaN,0.0,0,NaN,0,NaN,0,20110119,201412,서울특별시 동작구 상도동 302-28,10,일반,20080826.0,20080826.0,20110119,20190530
2,11320-100182241,집합,표제부,0,주건축물,208.00,81.085,38.98,268.165,268.165,128.93,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,(철근)콘크리트,10.40,4,0,0,0,0,0.0,268.165,0,0.0,0,0.0,3,0.00,1,0.0,20080721.0,NaN,0.0,0,NaN,0,NaN,0,20110202,201412,서울특별시 도봉구 쌍문동 380-24,4,일반,20080721.0,20080721.0,20110202,20190530
3,11440-32174,집합,표제부,0,주건축물,314.00,180.380,57.45,627.880,627.880,199.96,철근콘크리트구조,철근콘크리트구조,공동주택,다세대주택,(철근)콘크리트,평스라브,13.30,5,0,0,0,0,0.0,627.880,0,0.0,0,0.0,7,80.50,1,11.5,20050901.0,NaN,0.0,0,NaN,0,NaN,0,20110125,201412,서울특별시 마포구 성산동 200-260,8,일반,20050901.0,20050901.0,20110125,20171205
4,11740-100179671,집합,표제부,0,주건축물,0.00,564.010,0.00,6401.190,6401.190,0.00,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트,45.60,14,0,3,0,0,0.0,6401.190,0,0.0,0,0.0,0,0.00,0,0.0,20090311.0,NaN,0.0,0,NaN,0,NaN,0,20110122,201412,서울특별시 강동구 강일동 674-0,78,일반,20090311.0,20090311.0,20100918,20200428


In [4]:
bdinfo_df['기준년월_최초사용승인일'].fillna(bdinfo_df['주소별_최초생성일자'], inplace=True)

In [5]:
bdinfo_df['사용승인년도'] = bdinfo_df['기준년월_최초사용승인일'].apply(lambda x: int(str(x)[:4]))
bdinfo_df['사용승인년도'].describe()

count    935428.000000
mean       2000.218566
std          23.310637
min        1111.000000
25%        1992.000000
50%        2002.000000
75%        2008.000000
max        9920.000000
Name: 사용승인년도, dtype: float64

In [6]:
min_df = bdinfo_df[bdinfo_df['사용승인년도'] == bdinfo_df['사용승인년도'].min()]

In [7]:
bdinfo_df.loc[min_df.index, '사용승인년도'] = 1911
bdinfo_df['사용승인년도'].describe()

count    935428.000000
mean       2000.233105
std          23.003484
min        1900.000000
25%        1992.000000
50%        2002.000000
75%        2008.000000
max        9920.000000
Name: 사용승인년도, dtype: float64

In [8]:
bdinfo_df.to_csv('./국토교통부_건축물대장_표제부/bdinfo_baseline_maindata_inter_checkpoint_4.csv', index=False)

In [9]:
date_df = bdinfo_df[['지번주소', '기준년월', '사용승인년도']].drop_duplicates(subset=['지번주소', '기준년월'])
date_df.shape

(716540, 3)

In [10]:
date_df.to_csv('./추출한_건물정보/first_approved_date.csv', index=False)

In [11]:
del date_df

In [12]:
lot_num_df = bdinfo_df[['지번주소', '기준년월', '외필지_수']]

In [13]:
lot_num_df['총외필지수'] = lot_num_df.groupby(['지번주소', '기준년월']).transform('sum')['외필지_수']

In [14]:
lot_num_df = lot_num_df[['지번주소', '기준년월', '총외필지수']].drop_duplicates()
lot_num_df.shape

(716540, 3)

In [15]:
lot_num_df.to_csv('./추출한_건물정보/표제부상_외필지수.csv', index=False)

In [17]:
bdinfo_df['주_부속_구분'].value_counts()

주건축물     874418
부속건축물     61004
Name: 주_부속_구분, dtype: int64

In [18]:
bdinfo_df['부속건축물_여부'] = bdinfo_df['주_부속_구분'].apply(lambda x: 1 if x == '부속건축물' else 0)
bdinfo_df['부속건축물_여부'].value_counts()

0    874424
1     61004
Name: 부속건축물_여부, dtype: int64

In [19]:
sub_df = bdinfo_df[['지번주소', '기준년월', '부속건축물_여부']]
sub_df['부속건축물_수'] = sub_df.groupby(['지번주소', '기준년월']).transform('sum')['부속건축물_여부']
sub_df['부속건축물_수'].describe()

count    935428.000000
mean          1.446240
std           4.956807
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max          58.000000
Name: 부속건축물_수, dtype: float64

In [20]:
bdinfo_df['부속_건축물_수'].describe()

count    935428.000000
mean          0.083565
std           1.609497
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max         721.000000
Name: 부속_건축물_수, dtype: float64

In [21]:
max_df = bdinfo_df[bdinfo_df['부속_건축물_수'] == bdinfo_df['부속_건축물_수'].max()]
max_df.shape

(3, 52)

In [22]:
max_df

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인년도,부속건축물_여부
74886,11215-7446,일반,일반건축물,0,주건축물,0.0,80.0,0.0,193.72,141.49,0.0,벽돌구조,연와조,단독주택,주택,(철근)콘크리트,경사스라브위기와,0.0,2,1,0,0,721,0.0,193.72,0,0.0,0,0.0,0,0.0,0,0.0,19780721.0,NaN,0.0,0,NaN,0,NaN,0,20130723,201412,서울특별시 광진구 중곡동 604-5,1,일반,19780721.0,19780721.0,20130723,20180127,1978,0
198070,11215-7446,일반,일반건축물,0,주건축물,0.0,80.0,0.0,193.72,141.49,0.0,벽돌구조,연와조,단독주택,주택,(철근)콘크리트,경사스라브위기와,0.0,2,1,0,0,721,0.0,193.72,0,0.0,0,0.0,0,0.0,0,0.0,19780721.0,NaN,0.0,0,NaN,0,NaN,0,20160825,201612,서울특별시 광진구 중곡동 604-5,1,일반,19780721.0,19780721.0,20130723,20180127,1978,0
829517,11215-7446,일반,일반건축물,0,주건축물,0.0,80.0,0.0,193.72,141.49,0.0,벽돌구조,연와조,단독주택,주택,(철근)콘크리트,경사스라브위기와,0.0,2,1,0,0,721,0.0,193.72,0,0.0,0,0.0,0,0.0,0,0.0,19780721.0,NaN,0.0,0,NaN,0,NaN,0,20130723,201512,서울특별시 광진구 중곡동 604-5,1,일반,19780721.0,19780721.0,20130723,20180127,1978,0


In [23]:
sub_df.head()

,지번주소,기준년월,부속건축물_여부,부속건축물_수
0,서울특별시 마포구 서교동 484-16,201412,0,0
1,서울특별시 동작구 상도동 302-28,201412,0,0
2,서울특별시 도봉구 쌍문동 380-24,201412,0,0
3,서울특별시 마포구 성산동 200-260,201412,0,0
4,서울특별시 강동구 강일동 674-0,201412,0,7


In [24]:
sub_df.drop(columns=['부속건축물_여부'], inplace=True)
sub_df.drop_duplicates(inplace=True)

In [25]:
sub_df.shape

(716540, 3)

In [26]:
sub_df.to_csv('./추출한_건물정보/표제부상_부속건축물수.csv', index=False)

In [27]:
sub_area_df = bdinfo_df[['지번주소', '기준년월', '부속_건축물_면적(㎡)']]

In [28]:
sub_area_df['총부속건축물면적'] = sub_area_df.groupby(['지번주소', '기준년월']).transform('sum')['부속_건축물_면적(㎡)']

In [29]:
sub_area_df = sub_area_df[['지번주소', '기준년월', '총부속건축물면적']].drop_duplicates()
sub_area_df.shape

(716540, 3)

In [30]:
sub_area_df['총부속건축물면적'].describe()

count    7.165400e+05
mean     1.815766e+02
std      8.119293e+03
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      1.629110e+06
Name: 총부속건축물면적, dtype: float64

In [31]:
max_df = sub_area_df[sub_area_df['총부속건축물면적'] == sub_area_df['총부속건축물면적'].max()]
max_df.shape

(1, 3)

In [32]:
max_df

,지번주소,기준년월,총부속건축물면적
667145,서울특별시 강동구 강일동 114-0,202011,1629110.16


In [33]:
addr_df = bdinfo_df[bdinfo_df['지번주소'] == max_df['지번주소'].iloc[0]]
addr_df.shape

(36, 52)

In [34]:
addr_df

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인년도,부속건축물_여부
667145,11740-100277628,집합,표제부,0,부속건축물,0.0,19.16,0.0,69.29,0.00,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,게스트하우스,(철근)콘크리트,(철근)콘크리트평지붕,4.80,1,0,0,0,23,45253.06,69.29,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,0,일반,20200828.0,20200828.0,20200903,20200903,2020,1
668186,11740-100277060,집합,표제부,0,주건축물,0.0,104.74,0.0,155.11,121.34,0.0,철근콘크리트구조,철근콘크리트구조,제1종근린생활시설,사회적기업,(철근)콘크리트,(철근)콘크리트평지붕,4.80,1,1,0,0,23,45253.06,155.11,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,2,일반,20200828.0,20200828.0,20200903,20200903,2020,0
671370,11740-100277632,집합,표제부,0,부속건축물,0.0,0.00,0.0,28.46,28.46,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,경비실,(철근)콘크리트,(철근)콘크리트평지붕,4.80,1,0,0,0,23,45253.06,28.46,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,0,일반,20200828.0,20200828.0,20200903,20200903,2020,1
674533,11740-100277631,집합,표제부,0,부속건축물,0.0,22.62,0.0,22.62,22.62,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,경비실,(철근)콘크리트,(철근)콘크리트평지붕,4.80,1,0,0,0,23,45253.06,22.62,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,0,일반,20200828.0,20200828.0,20200903,20200903,2020,1
687206,11740-100277640,집합,표제부,0,부속건축물,0.0,0.00,0.0,182.78,0.00,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,열교환기실,(철근)콘크리트,(철근)콘크리트,4.80,0,2,0,0,23,45253.06,182.78,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,0,일반,20200828.0,20200828.0,20200903,20200903,2020,1
689341,11740-100276767,집합,표제부,0,주건축물,0.0,1731.70,0.0,15443.80,15443.80,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트평지붕,47.45,16,0,2,4,23,45253.06,15443.80,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,217,일반,20200828.0,20200828.0,20200903,20200903,2020,0
690405,11740-100277412,집합,표제부,0,주건축물,0.0,724.65,0.0,7969.40,7969.40,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트평지붕,50.35,17,0,0,3,23,45253.06,7969.40,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,97,일반,20200828.0,20200828.0,20200903,20200903,2020,0
695783,11740-100277192,집합,표제부,0,주건축물,0.0,1574.76,0.0,12938.69,12938.69,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트평지붕,47.45,16,0,1,4,23,45253.06,12938.69,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,219,일반,20200828.0,20200828.0,20200903,20200903,2020,0
705523,11740-100277627,집합,표제부,0,부속건축물,0.0,21.51,0.0,69.29,0.00,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,게스트하우스,(철근)콘크리트,(철근)콘크리트평지붕,4.80,1,0,0,0,23,45253.06,69.29,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,0,일반,20200828.0,20200828.0,20200903,20200903,2020,1
715251,11740-100276363,집합,표제부,0,주건축물,0.0,1174.04,0.0,9498.32,9498.32,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트평지붕,41.65,14,0,1,3,23,45253.06,9498.32,0,0.0,0,0.0,0,0.0,0,0.0,20200828.0,NaN,0.0,0,NaN,0,NaN,0,20200903,202011,서울특별시 강동구 강일동 114-0,125,일반,20200828.0,20200828.0,20200903,20200903,2020,0


In [35]:
bdinfo_df['총주차수'] = bdinfo_df['옥내_기계식_대수(대)'] + bdinfo_df['옥외_기계식_대수(대)'] + bdinfo_df['옥내_자주식_대수(대)'] + bdinfo_df['옥외_자주식_대수(대)']

In [36]:
parking_df = bdinfo_df[['지번주소', '기준년월', '총주차수']]

In [37]:
parking_df['기준년월_총주차수합'] = parking_df.groupby(['지번주소', '기준년월']).transform('sum')['총주차수']
parking_df['기준년월_총주차수합'].describe()

count    935428.000000
mean       1079.792660
std       14233.082277
min           0.000000
25%           0.000000
50%           5.000000
75%           8.000000
max      289110.000000
Name: 기준년월_총주차수합, dtype: float64

In [38]:
max_df = parking_df[parking_df['기준년월_총주차수합'] == parking_df['기준년월_총주차수합'].max()]
max_df

,지번주소,기준년월,총주차수,기준년월_총주차수합
3109,서울특별시 강서구 화곡동 1091-0,201412,4190,289110
4981,서울특별시 강서구 화곡동 1091-0,201412,4190,289110
7359,서울특별시 강서구 화곡동 1091-0,201412,4190,289110
7413,서울특별시 강서구 화곡동 1091-0,201412,4190,289110
7788,서울특별시 강서구 화곡동 1091-0,201412,4190,289110
...,...,...,...,...
924774,서울특별시 강서구 화곡동 1091-0,201512,4190,289110
924780,서울특별시 강서구 화곡동 1091-0,201512,4190,289110
925619,서울특별시 강서구 화곡동 1091-0,201512,4190,289110
928916,서울특별시 강서구 화곡동 1091-0,201512,4190,289110


In [39]:
addr_df = bdinfo_df[bdinfo_df['지번주소'] == max_df['지번주소'].iloc[0]]
addr_df

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인년도,부속건축물_여부,총주차수
3109,11500-1208,집합,표제부,0,주건축물,0.0,382.032,0.0,4268.896,3967.338,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,아파트(26세대),(철근)콘크리트,"(철근)콘크리트,경사지붕",40.078,14,1,1,0,0,0.0,4268.896,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20120710,201412,서울특별시 강서구 화곡동 1091-0,26,일반,20021031.0,20030709.0,20120710,20200427,2002,0,4190
4981,11500-1212,집합,표제부,0,주건축물,0.0,531.796,0.0,4330.455,4330.455,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,아파트(20세대),(철근)콘크리트,"(철근)콘크리트,경사지붕",29.928,10,0,1,0,0,0.0,4330.455,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20120710,201412,서울특별시 강서구 화곡동 1091-0,20,일반,20021031.0,20030709.0,20120710,20200427,2002,0,4190
7359,11500-1234,집합,표제부,0,주건축물,0.0,595.940,0.0,6987.740,6987.740,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,아파트(60세대),(철근)콘크리트,"(철근)콘크리트,경사지붕",42.752,15,0,2,0,0,0.0,6987.740,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20130416,201412,서울특별시 강서구 화곡동 1091-0,60,일반,20021031.0,20030709.0,20120710,20200427,2002,0,4190
7413,11500-1198,집합,표제부,0,주건축물,0.0,764.064,0.0,8479.464,8479.464,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,아파트(56세대),(철근)콘크리트,"(철근)콘크리트,경사지붕",40.078,14,0,2,0,0,0.0,8479.464,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20140212,201412,서울특별시 강서구 화곡동 1091-0,56,일반,20021031.0,20030709.0,20120710,20200427,2002,0,4190
7788,11500-1213,집합,표제부,0,주건축물,0.0,896.592,0.0,9068.720,9068.720,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,아파트(48세대),(철근)콘크리트,"(철근)콘크리트,경사지붕",35.168,12,0,2,0,0,0.0,9068.720,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20141013,201412,서울특별시 강서구 화곡동 1091-0,48,일반,20021031.0,20030709.0,20120710,20200427,2002,0,4190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
924774,11500-1217,집합,표제부,0,주건축물,0.0,896.592,0.0,8872.161,8872.164,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,아파트(47세대),(철근)콘크리트,"철근콘크리트조, 경사지붕",37.788,13,0,2,0,0,0.0,8872.161,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20120710,201512,서울특별시 강서구 화곡동 1091-0,47,일반,20021031.0,20030709.0,20120710,20200427,2002,0,4190
924780,11500-1260,집합,표제부,0,부속건축물,0.0,11520.000,0.0,11520.000,11520.000,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,경비실,(철근)콘크리트,(철근)콘크리트,0.000,1,0,0,0,0,0.0,11520.000,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20120710,201512,서울특별시 강서구 화곡동 1091-0,0,일반,20021031.0,20030709.0,20120710,20200427,2002,1,4190
925619,11500-1229,집합,표제부,0,주건축물,0.0,847.520,0.0,8258.432,8258.432,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,아파트(48세대),(철근)콘크리트,"(철근)콘크리트,경사지붕",34.948,12,0,2,0,0,0.0,8258.432,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20130416,201512,서울특별시 강서구 화곡동 1091-0,48,일반,20021031.0,20030709.0,20120710,20200427,2002,0,4190
928916,11500-1228,집합,표제부,0,주건축물,0.0,847.520,0.0,8258.432,8258.432,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,아파트(48세대),(철근)콘크리트,"(철근)콘크리트,경사지붕",34.948,12,0,2,0,0,0.0,8258.432,0,0.0,0,0.0,3536,0.0,654,0.0,20021031.0,NaN,0.0,0,NaN,0,NaN,0,20120710,201512,서울특별시 강서구 화곡동 1091-0,48,일반,20021031.0,20030709.0,20120710,20200427,2002,0,4190


In [40]:
parking_df.drop(columns=['기준년월_총주차수합'], inplace=True)

In [41]:
parking_df.head()

,지번주소,기준년월,총주차수
0,서울특별시 마포구 서교동 484-16,201412,4
1,서울특별시 동작구 상도동 302-28,201412,10
2,서울특별시 도봉구 쌍문동 380-24,201412,4
3,서울특별시 마포구 성산동 200-260,201412,8
4,서울특별시 강동구 강일동 674-0,201412,0


In [48]:
parking_df['기준년월_최대주차수'] = parking_df.groupby(['지번주소', '기준년월']).transform('max')['총주차수']
parking_df['기준년월_최대주차수'].describe()

count    935428.000000
mean         60.672129
std         522.471797
min           0.000000
25%           0.000000
50%           5.000000
75%           8.000000
max       16430.000000
Name: 기준년월_최대주차수, dtype: float64

In [49]:
max_df = parking_df[parking_df['기준년월_최대주차수'] == parking_df['기준년월_최대주차수'].max()]
max_df.shape

(203, 4)

In [50]:
max_df

,지번주소,기준년월,총주차수,기준년월_최대주차수
15789,서울특별시 강서구 마곡동 743-4,201412,0,16430
16034,서울특별시 강서구 마곡동 743-4,201412,16430,16430
17836,서울특별시 강서구 마곡동 743-4,201412,0,16430
26021,서울특별시 강서구 마곡동 743-4,201412,0,16430
26022,서울특별시 강서구 마곡동 743-4,201412,0,16430
26025,서울특별시 강서구 마곡동 743-4,201412,0,16430
28217,서울특별시 강서구 마곡동 743-4,201412,0,16430
30169,서울특별시 강서구 마곡동 743-4,201412,0,16430
32237,서울특별시 강서구 마곡동 743-4,201412,0,16430
38429,서울특별시 강서구 마곡동 743-4,201412,0,16430


In [47]:
parking_df.loc[max_df.index, '총주차수'] = 5

In [51]:
addr_df = bdinfo_df[bdinfo_df['지번주소'] == max_df['지번주소'].iloc[0]]
addr_df.shape

(203, 53)

In [52]:
addr_df

,관리_건축물대장_PK,대장_구분,대장_종류,외필지_수,주_부속_구분,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),구조,기타_구조,주_용도,기타_용도,지붕,기타_지붕,높이(m),지상_층_수,지하_층_수,승용_승강기_수,비상용_승강기_수,부속_건축물_수,부속_건축물_면적(㎡),총_동_연면적(㎡),옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,사용승인년도,부속건축물_여부,총주차수
15789,11500-100243159,집합,표제부,0,부속건축물,0.0,0.00,0.0,119.56,0.00,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,작은도서관,(철근)콘크리트,(철근)콘크리트평지붕,0.00,3,0,0,0,0,0.0,119.56,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,0,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,0,일반,20140612.0,20140612.0,20141021,20200427,2014,1,0
16034,11500-100242346,집합,표제부,0,주건축물,0.0,879.88,0.0,7802.91,7480.09,0.0,철근콘크리트구조,철근콘크리트구조,업무시설,"업무시설,판매시설",(철근)콘크리트,(철근)콘크리트평지붕,42.60,10,2,0,2,0,0.0,7802.91,0,0.0,0,0.0,16430,65444.42,0,0.0,20140612.0,NaN,0.0,0,NaN,0,NaN,0,20141225,201412,서울특별시 강서구 마곡동 743-4,46,일반,20140612.0,20140612.0,20141021,20200427,2014,0,16430
17836,11500-100242074,집합,표제부,0,주건축물,0.0,2072.74,0.0,16992.44,16992.44,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트평지붕,44.35,15,0,0,4,0,0.0,16992.44,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,86,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,130,일반,20140612.0,20140612.0,20141021,20200427,2014,0,0
26021,11500-100242706,집합,표제부,0,주건축물,0.0,808.57,0.0,7454.66,7454.66,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트평지붕,46.55,15,0,0,3,0,0.0,7454.66,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,86,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,78,일반,20140612.0,20140612.0,20141021,20200427,2014,0,0
26022,11500-100243157,집합,표제부,0,부속건축물,0.0,0.00,0.0,3404.87,0.00,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,"전기실,발전기실,기계실,펌프실,지열기계실,우수관리층",(철근)콘크리트,(철근)콘크리트평지붕,0.00,0,2,0,0,0,0.0,3404.87,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,0,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,0,일반,20140612.0,20140612.0,20141021,20200427,2014,1,0
26025,11500-100243161,집합,표제부,0,부속건축물,0.0,0.00,0.0,105.37,0.00,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,주민카페,(철근)콘크리트,(철근)콘크리트평지붕,0.00,3,0,0,0,0,0.0,105.37,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,0,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,0,일반,20140612.0,20140612.0,20141021,20200427,2014,1,0
28217,11500-100243156,집합,표제부,0,부속건축물,0.0,0.00,0.0,267.84,267.84,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,"관리사무소,MDF실,방재실",(철근)콘크리트,(철근)콘크리트평지붕,0.00,1,0,0,0,0,0.0,267.84,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,0,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,0,일반,20140612.0,20140612.0,20141021,20200427,2014,1,0
30169,11500-100243169,집합,표제부,0,부속건축물,0.0,15.63,0.0,15.63,15.63,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,경비실,(철근)콘크리트,(철근)콘크리트평지붕,3.60,1,0,0,0,0,0.0,15.63,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,0,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,0,일반,20140612.0,20140612.0,20141021,20200427,2014,1,0
32237,11500-100242918,집합,표제부,0,주건축물,0.0,873.50,0.0,9944.47,9944.47,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,공동주택(아파트),(철근)콘크리트,(철근)콘크리트평지붕,47.25,16,0,0,2,0,0.0,9944.47,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,86,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,78,일반,20140612.0,20140612.0,20141021,20200427,2014,0,0
38429,11500-100243166,집합,표제부,0,부속건축물,0.0,0.00,0.0,150.76,0.00,0.0,철근콘크리트구조,철근콘크리트구조,공동주택,경로당,(철근)콘크리트,(철근)콘크리트평지붕,0.00,1,0,0,0,0,0.0,150.76,0,0.0,0,0.0,0,0.00,0,0.0,20140612.0,NaN,0.0,0,NaN,0,NaN,0,20141021,201412,서울특별시 강서구 마곡동 743-4,0,일반,20140612.0,20140612.0,20141021,20200427,2014,1,0


In [ ]:
parking_df.drop(columns=['총주차수'], inplace=True)
parking_df.drop_duplicates(inplace=True)